# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770766952492                   -0.53    9.0    190ms
  2   -2.772142252305       -2.86       -1.30    1.0    106ms
  3   -2.772170178242       -4.55       -2.60    1.0    120ms
  4   -2.772170710556       -6.27       -3.59    2.0    125ms
  5   -2.772170722456       -7.92       -4.16    1.0    114ms
  6   -2.772170722961       -9.30       -4.98    1.0    129ms
  7   -2.772170723014      -10.28       -5.45    2.0    139ms
  8   -2.772170723015      -12.08       -6.23    1.0    123ms
  9   -2.772170723015      -13.70       -6.98    1.0    124ms
 10   -2.772170723015      -14.75       -7.59    2.0    188ms
 11   -2.772170723015      -14.27       -8.61    1.0    541ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770884876566                   -0.53    9.0    223ms
  2   -2.7

1.773557997262529

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770766232806                   -0.52    9.0    186ms
  2   -2.772059915207       -2.89       -1.33    1.0    121ms
  3   -2.772082874017       -4.64       -2.41    1.0    111ms
  4   -2.772083337409       -6.33       -3.11    1.0    116ms
  5   -2.772083417677       -7.10       -4.76    2.0    131ms
  6   -2.772083417808       -9.88       -4.94    2.0    134ms
  7   -2.772083417809      -11.98       -5.55    1.0    120ms
  8   -2.772083417811      -11.75       -6.44    2.0    598ms
  9   -2.772083417811      -13.56       -7.33    1.0    114ms
 10   -2.772083417811      -14.40       -7.69    1.0    118ms
 11   -2.772083417811   +  -13.94       -8.19    2.0    161ms

Polarizability via ForwardDiff:       1.7725349676481679
Polarizability via finite difference: 1.773557997262529
